In [80]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import datasets
import torch
import numpy as np

In [83]:
torch.cuda.empty_cache()

In [85]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    3369 MB |    6475 MB |   37271 GB |   37268 GB |
|       from large pool |    3365 MB |    6471 MB |   37263 GB |   37260 GB |
|       from small pool |       3 MB |      11 MB |       7 GB |       7 GB |
|---------------------------------------------------------------------------|
| Active memory         |    3369 MB |    6475 MB |   37271 GB |   37268 GB |
|       from large pool |    3365 MB |    6471 MB |   37263 GB |

In [69]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU: ', torch.cuda.get_device_name(0))
    
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device('cpu')

There are 1 GPU(s) available.
We will use the GPU:  GeForce RTX 2080 Ti


In [70]:
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')
model = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert', num_labels = 3)

In [71]:
features = datasets.Features({'text': datasets.Value('string'), 'labels': datasets.ClassLabel(num_classes = 3, names = [-1, 0, 1])})


In [72]:
dataset = datasets.load_dataset('/home/xic023/DSC180A-Methodology-5/data/out/', features = features)

Using custom data configuration out-1e5e62bdb76f9f1b
Found cached dataset csv (/tmp/xdg-cache/huggingface/datasets/csv/out-1e5e62bdb76f9f1b/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/2 [00:00<?, ?it/s]

In [73]:
train = dataset['train']
test = dataset['test']

In [74]:
train['labels']

[2,
 2,
 2,
 1,
 2,
 0,
 2,
 1,
 2,
 2,
 2,
 2,
 0,
 1,
 1,
 1,
 0,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 0,
 0,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 1,
 1,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 1,
 2,
 1,
 0,
 2,
 1,
 0,
 1,
 2,
 0,
 0,
 2,
 2,
 2,
 1,
 0,
 1,
 2,
 2,
 1,
 0,
 1,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 1,
 2,
 2,
 1,
 0,
 0,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 0,
 2,
 1,
 0,
 0,
 2,
 0,
 2,
 2,
 1,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 1,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 0,
 2,
 0,
 2,
 1,
 0,
 1,
 1,
 2,
 2,
 2,
 0,
 1,
 2,
 0,
 1,
 0,
 0,
 2,
 2,
 0,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 0,
 0,
 1,
 2,
 2,
 1,
 2,
 2,
 0,
 2,
 2,
 1,
 0,
 1,
 0,
 2,
 0,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 0,
 2,
 1,
 0,


In [75]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding = 'max_length', truncation = True)

tokenized_train = train.map(tokenize_function, batched = True)
tokenized_test = test.map(tokenize_function, batched = True)

Loading cached processed dataset at /tmp/xdg-cache/huggingface/datasets/csv/out-1e5e62bdb76f9f1b/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-20e6ec78fad2a35c.arrow
Loading cached processed dataset at /tmp/xdg-cache/huggingface/datasets/csv/out-1e5e62bdb76f9f1b/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-7695dcef0700fd20.arrow


In [76]:
args = {
    'output_dir': '/home/xic023/DSC180A-Methodology-5/results',
    'evaluation_strategy': 'epoch',
    'num_train_epochs': 1,
    'log_level': 'error',
    'report_to': 'none'
}

In [77]:
import evaluate

metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits ,axis = -1)
    return metric.compute(predictions = predictions, references = labels)

In [78]:
tokenized_test.features

{'text': Value(dtype='string', id=None),
 'labels': ClassLabel(names=[-1, 0, 1], id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [79]:
from transformers import TrainingArguments, Trainer, logging

logging.set_verbosity_error()

training_args = TrainingArguments(per_device_train_batch_size = 4, **args)
trainer = Trainer(model = model,
                  args = training_args,
                  train_dataset = tokenized_train,
                  eval_dataset = tokenized_test,
                  tokenizer = tokenizer,
                  compute_metrics = compute_metrics)
trainer.train()

/home/xic023/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyboardInterrupt: 

In [38]:
testdata = datasets.load_dataset('/home/xic023/DSC180A-Methodology-5/test/testdata/', features = features)

Using custom data configuration testdata-364d40fff1df3150


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /tmp/xdg-cache/huggingface/datasets/csv/testdata-364d40fff1df3150/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [39]:
testdata = testdata.map(tokenize_function, batched = True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [64]:
out = []
pred = trainer.predict(testdata['test']).predictions
for i in pred:
    out.append(int(np.where(i == max(i))[0]) - 1)

In [67]:
out

[1, 1, 1]